# LangGraph Store Integration Test

Test using AsyncScyllaDBStore without context manager for LangGraph integration.

## Setup

In [ ]:
import asyncio
from vertector_scylladbstore import AsyncScyllaDBStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv

load_dotenv()
print("✓ Imports successful")

## Initialize Store Without Context Manager

In [ ]:
# Initialize embeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
index_config = {"dims": 768, "embed": embeddings, "fields": ["$"]}

# Create store WITHOUT context manager (for LangGraph)
store_cm = AsyncScyllaDBStore.from_contact_points(
    contact_points=["127.0.0.1"],
    keyspace="langgraph_test",
    index=index_config,
    qdrant_url="http://localhost:6333",
)

# Extract store instance from async context manager
store = await store_cm.__aenter__()
await store.setup()

print("✓ Store initialized successfully")
print(f"  Keyspace: {store.keyspace}")
print(f"  Qdrant collection: {store.qdrant_collection}")

## Test Basic Operations

In [ ]:
# Test PUT operation
await store.aput(
    namespace=("users", "user_123"),
    key="profile",
    value={
        "name": "Alice",
        "preferences": "Likes Python and AI",
        "context": "Working on LangGraph integration"
    },
    wait_for_vector_sync=True  # Ensure immediate searchability
)

print("✓ PUT: Stored user profile")

In [ ]:
# Test GET operation
result = await store.aget(
    namespace=("users", "user_123"),
    key="profile"
)

print("✓ GET: Retrieved user profile")
print(f"  Name: {result.value['name']}")
print(f"  Preferences: {result.value['preferences']}")

In [ ]:
# Test SEARCH operation (semantic)
results = await store.asearch(
    ("users",),
    query="programming and artificial intelligence",
    limit=5
)

print(f"✓ SEARCH: Found {len(results)} results")
for item in results:
    print(f"  - {item.namespace} / {item.key}: {item.value.get('name')} (score: {item.score:.4f})")

## Simulate LangGraph Usage Pattern

In a real LangGraph application, the store would be used across multiple graph invocations.

In [ ]:
async def simulate_graph_invocation(store, user_id: str, conversation_turn: int):
    """Simulate a single graph invocation."""
    
    # Store conversation memory
    await store.aput(
        namespace=("conversations", user_id),
        key=f"turn_{conversation_turn}",
        value={
            "turn": conversation_turn,
            "user_message": f"What can you help me with in turn {conversation_turn}?",
            "assistant_response": f"I can help you with various tasks! (Turn {conversation_turn})",
            "context": "LangGraph integration test"
        },
        wait_for_vector_sync=True
    )
    
    # Retrieve conversation history
    history = await store.asearch(
        ("conversations", user_id),
        query="conversation history",
        limit=10
    )
    
    return len(history)

# Simulate 5 graph invocations
print("Simulating LangGraph invocations...")
for i in range(1, 6):
    count = await simulate_graph_invocation(store, "alice_001", i)
    print(f"  Turn {i}: {count} conversation turns in history")

print("\n✓ Multiple graph invocations completed successfully")

## Test Store Persistence Across Cells

The store should remain active and usable across notebook cells.

In [ ]:
# Verify store is still accessible
namespaces = await store.alist_namespaces(prefix=("conversations",))

print(f"✓ Store is still active")
print(f"  Active conversation namespaces: {len(namespaces)}")
for ns in namespaces:
    print(f"    - {ns}")

## Check Cache Performance

In [ ]:
# Get embedding cache statistics
cache_stats = store.get_embedding_cache_stats()

print("📊 Embedding Cache Statistics:")
print(f"  Cache size: {cache_stats['size']}/{cache_stats['max_size']}")
print(f"  Cache hits: {cache_stats['hits']}")
print(f"  Cache misses: {cache_stats['misses']}")
print(f"  Hit rate: {cache_stats['hit_rate']:.1%}")

## Cleanup Test Data

In [ ]:
# Clean up test data
async def cleanup_test_data(store):
    """Clean up all test data."""
    deleted_count = 0
    
    # Delete user profiles
    try:
        await store.adelete(("users", "user_123"), "profile")
        deleted_count += 1
    except:
        pass
    
    # Delete conversation history
    conversations = await store.asearch(("conversations",), limit=100)
    for item in conversations:
        await store.adelete(item.namespace, item.key)
        deleted_count += 1
    
    return deleted_count

count = await cleanup_test_data(store)
print(f"✓ Cleaned up {count} test items")

## Summary

### Key Findings:

1. **✅ No context manager needed** - Store can be initialized and used directly
2. **✅ Long-lived instance** - Store persists across multiple operations
3. **✅ No cleanup required** - BaseStore interface doesn't require explicit close/stop
4. **✅ LangGraph compatible** - Store can be passed to `graph.compile(store=store)`

### Usage Pattern for LangGraph:

```python
# At application startup
store_cm = AsyncScyllaDBStore.from_contact_points(...)
store = await store_cm.__aenter__()
await store.setup()

# Use in LangGraph
graph = builder.compile(checkpointer=checkpointer, store=store)

# Store remains active for the lifetime of the application
# No explicit cleanup needed!
```